In [2]:
import pandas as pd
from qdrant_client import models, QdrantClient
from openai import OpenAI
import clip
from PIL import Image
import torch
from transformers import  CLIPModel, CLIPTextModel, AutoProcessor, AutoTokenizer, CLIPProcessor


In [3]:
# Read the dataset
# download the dataset from this page

df = pd.read_csv('./csv/styles.csv')
data = df.to_dict('records') # Get only 700 records. More records will make it slower to index


In [4]:
# get image feature
def get_features_from_image_path(url):
  model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
  processor = processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
  image = Image.open(url)
  inputs = processor(images=image, return_tensors="pt")
  image_features = model.get_image_features(**inputs)
  return image_features



In [5]:
# get text features
def get_prompt_features(prompt):
  model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
  processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
  tokens = processor(
    text=prompt,
    padding=True,
    images=None,
    return_tensors='pt'
)
  text_emb = model.get_text_features(
    **tokens
)

  return text_emb

In [6]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance
# Create collection to store wines
qdrant.create_collection(
    collection_name="top_fashion",
    vectors_config=models.VectorParams(
        size= 512,# vectore size clip
        distance=models.Distance.COSINE
    )
)


True

In [7]:
#I put the data records as the list of dictionaly to import to the vectoreDB as payloads
df = pd.read_csv('./csv/styles.csv')
data = df.to_dict('records')
d = []
for i,doc in enumerate(data):
  d.append(doc)



In [8]:
# write a dataloader to load the data in batch
# the original dataloader in python merge dictionary types of data when it makes a batch
# then I pass payloads seperatly out of the dataloader


from torch.utils.data import Dataset
class CustomDataSet(Dataset):
    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)
        self.data = df.to_dict('records')
        self.doc = doc

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        image_paths = './img/'+str(self.data[index]['id'])+'.jpg'
        image = get_features_from_image_path(image_paths)[0]
        label = index

        return image, label



csv_file_path = './csv/styles.csv'
train_data_object = CustomDataSet(csv_file_path)
train_loader = torch.utils.data.DataLoader(train_data_object,
        batch_size=4,
        shuffle=False)



In [10]:
# add samples in batch mode to the dataset
# I have limited the data iteration to save time here for demo
import numpy as np
image_loader = iter( train_loader)
for i in range(100):
  image_feature, idx = next(image_loader)
  
    
    
  data = list(np.array(d)[idx])
  qdrant.upsert(
      collection_name="top_fashion",
      points=models.Batch(
          ids=idx.tolist(),
          vectors=image_feature.tolist(),
          payloads=data # it must be a list of dicts otherwise it returns an error
      )
  )

In [ ]:
## add the samples one by one to the database instead of batch
## vectorize!
# for idx, doc in enumerate(data):
#   image_paths = '/content/drive/MyDrive/Colab Notebooks/img/'+str(data[idx]['id'])+'.jpg'
#   qdrant.upload_points(
#       collection_name="top_fashion",
#       points=[
#           models.PointStruct(
#               id=idx,
#               vector= get_features_from_image_path(image_paths)[0].tolist(),
#               payload=doc,
#           )
#       ]
#   )

In [33]:
user_prompt = " I'm searching for a men summer jean"
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_fashion",
    query_vector=get_prompt_features(user_prompt)[0].tolist(),
    limit=3
)
for hit in hits:
    print(hit.payload, "score:", hit.score, hit.id)
im = Image.open('./img/'+str(hits[0].payload['id'])+'.jpg')
im.save("result.jpg")
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]



{'id': 39386, 'gender': 'Men', 'masterCategory': 'Apparel', 'subCategory': 'Bottomwear', 'articleType': 'Jeans', 'baseColour': 'Blue', 'season': 'Summer', 'year': 2012.0, 'usage': 'Casual', 'productDisplayName': 'Peter England Men Party Blue Jeans', 'Unnamed: 10': nan} score: 0.31613736558058403 1
{'id': 21379, 'gender': 'Men', 'masterCategory': 'Apparel', 'subCategory': 'Bottomwear', 'articleType': 'Track Pants', 'baseColour': 'Black', 'season': 'Fall', 'year': 2011.0, 'usage': 'Casual', 'productDisplayName': 'Manchester United Men Solid Black Track Pants', 'Unnamed: 10': nan} score: 0.2499250589587359 3
{'id': 15970, 'gender': 'Men', 'masterCategory': 'Apparel', 'subCategory': 'Topwear', 'articleType': 'Shirts', 'baseColour': 'Navy Blue', 'season': 'Fall', 'year': 2011.0, 'usage': 'Casual', 'productDisplayName': 'Turtle Check Men Navy Blue Shirt', 'Unnamed: 10': nan} score: 0.23541226837191617 0


In [34]:

# from google.colab import userdata

client = OpenAI(
    api_key = "your openAI api-key",
)

In [35]:
style_prompt = " I'm a white tall man."
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are chatbot, a facshion stylist. Your top priority is to help guide users into pairing amazing outfit for a person "+style_prompt +"with the item that the customer is searching for."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(search_results[0])}

    ]
)

print(completion.choices[0].message.content)

PROMPT = completion.choices[0].message.content + "show them on a body like" + style_prompt

response = client.images.generate(
    model="dall-e-3",
    prompt=PROMPT,
)

print(response.data[0].url)



Great choice! For a summer outfit featuring blue jeans, here's a stylish pairing:

### Outfit Idea:
- **Jeans:** Peter England Men Party Blue Jeans (or any slim-fit dark blue or light wash jeans)
  
### Top:
- **Shirt:** A lightweight, short-sleeve button-up shirt in a light color like white, pale blue, or pastel shades. A subtle pattern like checks or stripes can add character.

### Footwear:
- **Shoes:** White sneakers or casual loafers will keep the look fresh and comfortable for summer.

### Accessories:
- **Watch:** A classic leather or canvas strap watch.
- **Sunglasses:** A pair of trendy sunglasses for sun protection and style.
- **Belt:** A brown or navy leather belt to match your footwear.

### Optional Layer:
- If it gets cool in the evening, consider adding a light denim jacket or a casual blazer to elevate the look.

This outfit is not only stylish but also comfortable for a summer outing. Enjoy your summer!
https://oaidalleapiprodscus.blob.core.windows.net/private/org-Q4z